In [ ]:
import torch
import torch.nn as nn

# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))


In [46]:
# 1번 레이어 : 합성곱층(Convolutional layer)
#합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
print(conv1)

# 2번 레이어 : 합성곱층(Convolutional layer)
#합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

#맥스풀링(kernel_size=2, stride=2))
pool = nn.MaxPool2d(2)
print(pool)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [47]:
out = conv1(inputs)
print(out.shape)
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)
#현재 out의 크기는 1 × 64 × 7 × 7입니다.
# out의 첫번째 차원이 몇인지 출력해보겠습니다.
print(out.size(0))
#out의 첫번째 차원은 1입니다.
#두번째 차원이 몇인지 출력해보겠습니다.
print(out.size(1))
print(out.size(2))
print(out.size(3))

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
1
64
7
7


In [48]:
#.view() 를 사용하여 텐서를 펼치는 작업
out = out.view(out.size(0), -1)
print(out.shape)
#배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있습니다.
# 이제 이에 대해서 전결합층(Fully-Connteced layer)를 통과시켜보겠습니다.
# 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환합니다.

torch.Size([1, 3136])


In [49]:
print(out.shape[1])
fc = nn.Linear(out.shape[1], 10) # # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

3136
torch.Size([1, 10])


In [2]:
# 3. CNN으로 MNIST 분류하기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epoch = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',#다운로드 경로
                          train=True,
                          transform = transforms.ToTensor(),#텐서로 변환
                          download=True )

mnist_test = dsets.MNIST(root='MNIST_data/',#다운로드 경로
                          train=False,
                          transform = transforms.ToTensor(),#텐서로 변환
                          download=True )

data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)



In [3]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #layer 1
        #ImgIn shape = (?, 28,28, 1)
        #  conv --> (?, 28,28,32)
        #  pool --> (?, 14,14,32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        #layer 2
        #ImgIn shape = (?, 14, 14, 32)
        #  conv --> (?, 14,14,64)
        #  pool --> (?, 7,7,64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        #fully connected layer
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)

        #
        torch.nn.init.xavier_uniform(self.fc.weight)

    def forward(self, x ):
        out =self.layer1(x)
        out =self.layer2(out)
        out = out.view(out.size(0), -1) #fully connected layer를 위해 flatten
        out = self.fc(out)
        return out

In [8]:
#define model
model = CNN().to(device)

criterian = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('total batch count = {}'.format(total_batch))
print("training epoch count", training_epoch)

for epoch in range(training_epoch):
    avg_cost = 0
    for X, Y in data_loader: #미니배치 단위로 꺼내온다.  X는 미니배치, Y는 레이블
        #image is already size of 28*28 , no reshape
        #label is not one-hot encoded
        X=X.to(device)
        Y=Y.to(device)
        #print(X.shape, Y.shape)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterian(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch

    print('epoch: {:>4} cost={:>.9}'.format(epoch + 1, avg_cost))



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_20732/3232897716.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc.weight)


total batch count = 600
training epoch count 15
epoch:    1 cost=0.212764308
epoch:    2 cost=0.0589677915
epoch:    3 cost=0.0425912328
epoch:    4 cost=0.0342711508
epoch:    5 cost=0.0284705404
epoch:    6 cost=0.0231754892
epoch:    7 cost=0.0196241047
epoch:    8 cost=0.0164618138
epoch:    9 cost=0.0142560797
epoch:   10 cost=0.0116690882
epoch:   11 cost=0.0100624152
epoch:   12 cost=0.0094806226
epoch:   13 cost=0.0081340177
epoch:   14 cost=0.00661981525
epoch:   15 cost=0.00724106608


In [10]:
#테스트, 학습을 진행하지 않으므로 torch.no_grad
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim=1 ) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())


Accuracy: 0.9873999953269958
